In [8]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('S30 ETL Assignment.db')
df = pd.read_sql_query("""SELECT 
    c.customer_id,
    i.item_name,
    SUM(o.quantity) AS total_quantity
FROM 
    customers c
LEFT JOIN 
    sales s ON c.customer_id = s.customer_id
LEFT JOIN 
    orders o ON s.sales_id = o.sales_id
LEFT JOIN 
    items i ON o.item_id = i.item_id
WHERE 
    c.age BETWEEN 18 AND 35
GROUP BY 
    c.customer_id, i.item_name
HAVING 
    SUM(o.quantity) > 0;
""", conn)
print(df)
conn.close()

    customer_id item_name  total_quantity
0             5         x               3
1             5         y               3
2             5         z               4
3             7         x              14
4             7         y              19
..          ...       ...             ...
65           97         y               2
66           97         z               3
67          100         x               1
68          100         y               1
69          100         z               5

[70 rows x 3 columns]


In [ ]:
import sqlite3
import pandas as pd

# open connection
conn = sqlite3.connect('S30 ETL Assignment.db')

customer_df = pd.read_sql_query("SELECT * FROM customers", conn)
sales_df = pd.read_sql_query("SELECT * FROM sales", conn)
orders_df = pd.read_sql_query("SELECT * FROM orders", conn)
items_df = pd.read_sql_query("SELECT * FROM items", conn)

filtered_customers = customer_df[customer_df['age'].between(18, 35)]

merged_df = (
    filtered_customers
    .merge(sales_df, on='customer_id')
    .merge(orders_df, on='sales_id')
    .merge(items_df, on='item_id')
)

result_df = (
    merged_df
    .groupby(['customer_id', 'item_name'])['quantity']
    .sum()
    .reset_index()
)
result_df = result_df[result_df['quantity'] > 0]
result_df['quantity'] = result_df['quantity'].astype(int)

# save to csv
result_df.to_csv('output.csv', sep=';', index=False)

# close connection
conn.close()
